In [1]:
pip install --upgrade pyod


[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install combo


[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install torch

     -------------------------------------- 198.6/198.6 MB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 4.9 MB/s eta 0:00:00
     -------------------------------------- 172.0/172.0 kB 5.1 MB/s eta 0:00:00
     -------------------------------------- 536.2/536.2 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0

[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [74]:
pip install torch_geometric

     ---------------------------------------- 1.1/1.1 MB 4.3 MB/s eta 0:00:00
     -------------------------------------- 370.5/370.5 kB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 76.4/76.4 kB ? eta 0:00:00
     ---------------------------------------- 50.4/50.4 kB 2.5 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import json
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import ruptures as rpt
from collections import Counter

In [4]:
columns = ['sensors.battery.temp',
       'sensors.battery.percent', 'sensors.battery.solarmv',
       'sensors.battery.current', 'sensors.battery.voltage',
       'sensors.battery.solarma', 'sensors.battery.charging',
        'solar_power', 'battery_power','power_difference']

In [5]:
# Reading back and setting the MultiIndex
labelled_data = pd.read_feather('MTDataBatch1\Labelled Data\Labelled Batch 1 - Updated.feather')
labelled_data.set_index(['DeviceId', 'time'], inplace=True)
# labelled_data

In [6]:
labelled_data

Id  \
DeviceId        time                                   
015262001277846 2021-04-21 03:18:26+10:00      10046   
                2021-04-21 03:17:52+10:00      10049   
                2021-04-21 03:27:29+10:00      15189   
                2021-04-21 03:28:29+10:00      15190   
                2021-04-21 03:21:30+10:00      15191   
...                                              ...   
015262001276442 2021-11-22 13:01:23+11:00  195389569   
                2021-11-22 13:04:17+11:00  195391320   
                2021-11-23 13:05:02+11:00  196545125   
                2021-11-24 13:05:36+11:00  197957676   
                2021-11-24 13:05:02+11:00  197957683   

                                                                    CreatedAt  \
DeviceId        time                                                            
015262001277846 2021-04-21 03:18:26+10:00  2021-04-20 17:18:57.3601057 +00:00   
                2021-04-21 03:17:52+10:00  2021-04-20 17:18:57.9206698 +00:00   
                2021-04-21 03:27:29+10:00  2021-04-20 17:30:00.0973275 +00:00   
                2021-04-21 03:28:29+10:00  2021-04-20 17:30:00.5204188 +00:00   
                2021-04-21 03:21:30+10:00  2021-04-20 17:30:00.6604193 +00:00   
...                                                                       ...   
015262001276442 2021-11-22 13:01:23+11:00  2021-11-22 02:02:07.3194074 +00:00   
                2021-11-22 13:04:17+11:00  2021-11-22 02:04:57.8307232 +00:00   
                2021-11-23 13:05:02+11:00  2021-11-23 02:05:44.6179891 +00:00   
                2021-11-24 13:05:36+11:00  2021-11-24 02:06:18.7314233 +00:00   
                2021-11-24 13:05:02+11:00  2021-11-24 02:06:19.3460223 +00:00   

                                                                    UpdatedAt  \
DeviceId        time                                                            
015262001277846 2021-04-21 03:18:26+10:00  2021-04-20 17:18:57.4751133 +00:00   
                2021-04-21 03:17:52+10:00  2021-04-20 17:18:58.0006530 +00:00   
                2021-04-21 03:27:29+10:00  2021-04-20 17:30:00.1973672 +00:00   
                2021-04-21 03:28:29+10:00  2021-04-20 17:30:00.6554035 +00:00   
                2021-04-21 03:21:30+10:00  2021-04-20 17:30:00.7254268 +00:00   
...                                                                       ...   
015262001276442 2021-11-22 13:01:23+11:00  2021-11-22 02:02:07.4609948 +00:00   
                2021-11-22 13:04:17+11:00  2021-11-22 02:04:57.9257593 +00:00   
                2021-11-23 13:05:02+11:00  2021-11-23 02:05:44.7707739 +00:00   
                2021-11-24 13:05:36+11:00  2021-11-24 02:06:18.8826478 +00:00   
                2021-11-24 13:05:02+11:00  2021-11-24 02:06:19.4060168 +00:00   

                                          State             timeStamp  \
DeviceId        time                                                    
015262001277846 2021-04-21 03:18:26+10:00     1  2021-04-20T17:18:26Z   
                2021-04-21 03:17:52+10:00     1  2021-04-20T17:17:52Z   
                2021-04-21 03:27:29+10:00     1  2021-04-20T17:27:29Z   
                2021-04-21 03:28:29+10:00     1  2021-04-20T17:28:29Z   
                2021-04-21 03:21:30+10:00     1  2021-04-20T17:21:30Z   
...                                         ...                   ...   
015262001276442 2021-11-22 13:01:23+11:00     1  2021-11-22T02:01:23Z   
                2021-11-22 13:04:17+11:00     1  2021-11-22T02:04:17Z   
                2021-11-23 13:05:02+11:00     1  2021-11-23T02:05:02Z   
                2021-11-24 13:05:36+11:00     1  2021-11-24T02:05:36Z   
                2021-11-24 13:05:02+11:00     1  2021-11-24T02:05:02Z   

                                             event  sensors.battery.extpower  \
DeviceId        time                                                           
015262001277846 2021-04-21 03:18:26+10:00   moving                     False   
                2021-04

In [7]:
labelled_data = labelled_data.sort_index(level=['DeviceId', 'time'])

In [8]:
numberOfDevices = len(labelled_data.groupby(level=0))
numberOfDevices

35

In [9]:
column_list = list(labelled_data.columns)
# for column in column_list:
#     print(column)
#     print(labelled_data[column].isnull().values.any())
#     print("-------------------")

In [10]:
len(labelled_data.columns)

31

In [11]:
train_size = int(len(labelled_data) * 0.80)
test_size = len(labelled_data) - train_size

In [12]:
# Training data : take all rows, and only device feature columns (From "sensors.battery.extpower" to "power_difference")
train_data = labelled_data.iloc[0:train_size,6:29]
test_data = labelled_data.iloc[train_size:len(labelled_data),6:29]
# Drop columns that have string values, or even NaN values 
# Manual Inspection proved them as non important for malfunction detectio.
train_data = train_data.drop('location.source', axis=1)
train_data = train_data.drop('location.numSats', axis=1)
train_data = train_data.drop('sensors.accelerometer.y', axis=1)
train_data = train_data.drop('sensors.accelerometer.x', axis=1)
train_data = train_data.drop('sensors.accelerometer.z', axis=1)
test_data = test_data.drop('location.source', axis=1)
test_data = test_data.drop('location.numSats', axis=1)
test_data = test_data.drop('sensors.accelerometer.y', axis=1)
test_data = test_data.drop('sensors.accelerometer.x', axis=1)
test_data = test_data.drop('sensors.accelerometer.z', axis=1)
# Labels is the array of values in "isDeviceWorking" -  convert to array of inetgers (0 or 1 values)
labels = labelled_data.iloc[:,29].values


In [13]:
# train_data_label = pd.DataFrame(labelled_data.iloc[0:train_size,29].values.astype(int), columns=['isDeviceWorking'])
# test_data_label = pd.DataFrame(labelled_data.iloc[train_size:len(labelled_data),29].values.astype(int), columns=['isDeviceWorking'])

In [14]:
len(train_data.columns)

18

In [15]:
# # Timestamps of devices that are working
# train_data_label.loc[train_data_label['isDeviceWorking'] == 1]

In [16]:
deviceWorkingLabels = labelled_data[['isDeviceWorking']].copy()
deviceWorkingLabels['isDeviceWorking'] = deviceWorkingLabels['isDeviceWorking'].astype(int)
deviceWorkingLabels

isDeviceWorking
DeviceId        time                                      
015262001090339 2021-06-11 12:27:45+10:00                1
                2021-06-11 12:34:17+10:00                1
                2021-06-11 16:34:25+10:00                1
                2021-06-11 20:34:39+10:00                1
                2021-06-12 00:35:02+10:00                1
...                                                    ...
015262001281806 2021-11-09 09:42:02+11:00                0
                2021-11-10 09:42:09+11:00                0
                2021-11-10 09:45:07+11:00                0
                2021-11-11 09:45:49+11:00                0
                2021-11-11 09:46:42+11:00                0

[1684083 rows x 1 columns]

In [17]:
train_data_label = deviceWorkingLabels.iloc[0:train_size,:]
test_data_label = deviceWorkingLabels.iloc[train_size:len(deviceWorkingLabels),:]

In [18]:
train_data_label

isDeviceWorking
DeviceId        time                                      
015262001090339 2021-06-11 12:27:45+10:00                1
                2021-06-11 12:34:17+10:00                1
                2021-06-11 16:34:25+10:00                1
                2021-06-11 20:34:39+10:00                1
                2021-06-12 00:35:02+10:00                1
...                                                    ...
015262001278133 2021-06-15 01:05:16+10:00                1
                2021-06-15 01:06:16+10:00                1
                2021-06-15 01:07:20+10:00                1
                2021-06-15 01:08:37+10:00                1
                2021-06-15 01:09:39+10:00                1

[1347266 rows x 1 columns]

In [19]:
# Scale column values (without normalizing the indexes)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1,1))

# Group by level 0, which is the first index corresponding to 'DeviceId'
# After applying the scaling function (which output a Numpy array), convert it to a dataframe

normalized_train_data = train_data.groupby(level=0).apply(lambda x : pd.DataFrame(scaler.fit_transform(x), columns=x.columns, index=x.index).round(5))
    
normalized_test_data = test_data.groupby(level=0).apply(lambda x : pd.DataFrame(scaler.fit_transform(x), columns=x.columns, index=x.index).round(5))

# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# scaler.fit(train_data)

# normalized_train_data = pd.DataFrame(scaler.transform(train_data))
# normalized_test_data = pd.DataFrame(scaler.transform(test_data))
normalized_test_data

sensors.battery.extpower  \
DeviceId        time                                                  
015262001278133 2021-06-15 01:10:39+10:00                      -1.0   
                2021-06-15 01:11:37+10:00                      -1.0   
                2021-06-15 01:12:40+10:00                      -1.0   
                2021-06-15 01:13:40+10:00                      -1.0   
                2021-06-15 01:14:42+10:00                      -1.0   
...                                                             ...   
015262001281806 2021-11-09 09:42:02+11:00                      -1.0   
                2021-11-10 09:42:09+11:00                      -1.0   
                2021-11-10 09:45:07+11:00                      -1.0   
                2021-11-11 09:45:49+11:00                      -1.0   
                2021-11-11 09:46:42+11:00                      -1.0   

                                           sensors.battery.temp  \
DeviceId        time                                              
015262001278133 2021-06-15 01:10:39+10:00              -0.46263   
                2021-06-15 01:11:37+10:00              -0.46263   
                2021-06-15 01:12:40+10:00              -0.46667   
                2021-06-15 01:13:40+10:00              -0.46667   
                2021-06-15 01:14:42+10:00              -0.46667   
...                                                         ...   
015262001281806 2021-11-09 09:42:02+11:00              -0.16606   
                2021-11-10 09:42:09+11:00              -0.16968   
                2021-11-10 09:45:07+11:00              -0.14079   
                2021-11-11 09:45:49+11:00              -0.04693   
                2021-11-11 09:46:42+11:00              -0.04693   

                                           sensors.battery.percent  \
DeviceId        time                                                 
015262001278133 2021-06-15 01:10:39+10:00                  1.00000   
                2021-06-15 01:11:37+10:00                  1.00000   
                2021-06-15 01:12:40+10:00                  1.00000   
                2021-06-15 01:13:40+10:00                  1.00000   
                2021-06-15 01:14:42+10:00                  1.00000   
...                                                            ...   
015262001281806 2021-11-09 09:42:02+11:00                 -0.97534   
                2021-11-10 09:42:09+11:00                 -0.98356   
                2021-11-10 09:45:07+11:00                 -0.99178   
                2021-11-11 09:45:49+11:00                 -1.00000   
                2021-11-11 09:46:42+11:00                 -0.99726   

                                           sensors.battery.solarmv  \
DeviceId        time                                                 
015262001278133 2021-06-15 01:10:39+10:00                 -1.00000   
                2021-06-15 01:11:37+10:00                 -0.97759   
                2021-06-15 01:12:40+10:00                 -1.00000   
                2021-06-15 01:13:40+10:00                 -1.00000   
                2021-06-15 01:14:42+10:00                 -1.00000   
...                                                            ...   
015262001281806 2021-11-09 09:42:02+11:00                 -0.38856   
                2021-11-10 09:42:09+11:00                 -0.22449   
                2021-11-10 09:45:07+11:00                 -0.24610   
                2021-11-11 09:45:49+11:00                 -0.13886   
                2021-11-11 09:46:42+11:00                 -0.13806   

                                           sensors.battery.current  \
DeviceId        time                                                 
015262001278133 2021-06-15 01:10:39+10:00                 -0.32571   
                2021-06-15 01:11:37+10:00                 -0.22286   
                2021-06-15 01:12:40+10:00                 -0.35429   
                2021-06-15 01:13:40+10:00                 -0.31429   
     

In [20]:
testdeviceIDs = set(normalized_test_data.index.get_level_values(0).tolist())
print(testdeviceIDs)

{'015262001278315', '015262001278513', '015262001278323', '015262001278133', '015262001278422', '015262001281806', '015262001278372'}


In [21]:
normalized_train_data[normalized_train_data.index.get_level_values('DeviceId') == '015262001277846']

sensors.battery.extpower  \
DeviceId        time                                                  
015262001277846 2021-04-21 03:17:52+10:00                      -1.0   
                2021-04-21 03:18:26+10:00                      -1.0   
                2021-04-21 03:19:31+10:00                      -1.0   
                2021-04-21 03:20:31+10:00                      -1.0   
                2021-04-21 03:21:30+10:00                      -1.0   
...                                                             ...   
                2021-05-30 07:30:27+10:00                      -1.0   
                2021-05-31 07:31:02+10:00                      -1.0   
                2021-05-31 07:31:46+10:00                      -1.0   
                2021-06-01 07:32:25+10:00                      -1.0   
                2021-06-02 07:32:31+10:00                      -1.0   

                                           sensors.battery.temp  \
DeviceId        time                                              
015262001277846 2021-04-21 03:17:52+10:00              -0.51194   
                2021-04-21 03:18:26+10:00              -0.51724   
                2021-04-21 03:19:31+10:00              -0.50663   
                2021-04-21 03:20:31+10:00              -0.50133   
                2021-04-21 03:21:30+10:00              -0.50663   
...                                                         ...   
                2021-05-30 07:30:27+10:00              -0.76658   
                2021-05-31 07:31:02+10:00              -0.80371   
                2021-05-31 07:31:46+10:00              -0.79841   
                2021-06-01 07:32:25+10:00              -0.97878   
                2021-06-02 07:32:31+10:00              -0.84085   

                                           sensors.battery.percent  \
DeviceId        time                                                 
015262001277846 2021-04-21 03:17:52+10:00                  0.98276   
                2021-04-21 03:18:26+10:00                  0.98707   
                2021-04-21 03:19:31+10:00                  0.99569   
                2021-04-21 03:20:31+10:00                  0.99569   
                2021-04-21 03:21:30+10:00                  0.98707   
...                                                            ...   
                2021-05-30 07:30:27+10:00                 -0.95690   
                2021-05-31 07:31:02+10:00                 -0.97414   
                2021-05-31 07:31:46+10:00                 -0.96983   
                2021-06-01 07:32:25+10:00                 -0.97414   
                2021-06-02 07:32:31+10:00                 -1.00000   

                                           sensors.battery.solarmv  \
DeviceId        time                                                 
015262001277846 2021-04-21 03:17:52+10:00                 -1.00000   
                2021-04-21 03:18:26+10:00                 -1.00000   
                2021-04-21 03:19:31+10:00                 -1.00000   
                2021-04-21 03:20:31+10:00                 -1.00000   
                2021-04-21 03:21:30+10:00                 -1.00000   
...                                                            ...   
                2021-05-30 07:30:27+10:00                 -0.69428   
                2021-05-31 07:31:02+10:00                 -0.58944   
                2021-05-31 07:31:46+10:00                 -0.58944   
                2021-06-01 07:32:25+10:00                 -0.78231   
                2021-06-02 07:32:31+10:00                 -0.75830   

                                           sensors.battery.current  \
DeviceId        time                                                 
015262001277846 2021-04-21 03:17:52+10:00                 -0.10811   
                2021-04-21 03:18:26+10:00                 -0.13514   
                2021-04-21 03:19:31+10:00                 -0.37838   
                2021-04-21 03:20:31+10:00                 -0.08108   
     

In [26]:
def get_feature_map(dataset):
    feature_list = list(dataset.columns)
    return feature_list

In [27]:
def get_fc_graph_struc(dataset):

    struc_map = {}

    feature_list = list(dataset.columns)

    for ft in feature_list:
        if ft not in struc_map:
            struc_map[ft] = []

        for other_ft in feature_list:
            if other_ft is not ft:
                struc_map[ft].append(other_ft)
    
    return struc_map

In [25]:
def build_loc_net(struc, all_features, feature_map=[]):

    index_feature_map = feature_map
    edge_indexes = [
        [],
        []
    ]
    for node_name, node_list in struc.items():
        if node_name not in all_features:
            continue

        if node_name not in index_feature_map:
            index_feature_map.append(node_name)
        
        p_index = index_feature_map.index(node_name)
        for child in node_list:
            if child not in all_features:
                continue

            if child not in index_feature_map:
                print(f'error: {child} not in index_feature_map')
                # index_feature_map.append(child)

            c_index = index_feature_map.index(child)
            # edge_indexes[0].append(p_index)
            # edge_indexes[1].append(c_index)
            edge_indexes[0].append(c_index)
            edge_indexes[1].append(p_index)
        

    
    return edge_indexes

In [32]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split, Subset

from sklearn.preprocessing import MinMaxScaler

In [33]:
feature_map = get_feature_map(normalized_train_data)
fc_struc = get_fc_graph_struc(normalized_train_data)
fc_edge_index = build_loc_net(fc_struc, list(normalized_train_data.columns), feature_map=feature_map)
fc_edge_index = torch.tensor(fc_edge_index, dtype = torch.long)

In [35]:
def construct_data(data, feature_map, labels=0):
    res = []

    for feature in feature_map:
        if feature in data.columns:
            res.append(data.loc[:, feature].values.tolist())
        else:
            print(feature, 'not exist in data')
    # append labels as last
    sample_n = len(res[0])

    if type(labels) == int:
        res.append([labels]*sample_n)
    elif len(labels) == sample_n:
        res.append(labels)

    return res

In [38]:
train_dataset_indata = construct_data(normalized_train_data, feature_map, labels=train_data_label.isDeviceWorking.tolist())
test_dataset_indata = construct_data(normalized_test_data, feature_map, labels=test_data_label.isDeviceWorking.tolist())

In [68]:
def get_loaders(train_dataset, seed, batch, val_ratio=0.1):
    dataset_len = int(len(train_dataset))
    train_use_len = int(dataset_len * (1 - val_ratio))
    val_use_len = int(dataset_len * val_ratio)
    val_start_index = random.randrange(train_use_len)
    indices = torch.arange(dataset_len)

    train_sub_indices = torch.cat([indices[:val_start_index], indices[val_start_index+val_use_len:]])
    train_subset = Subset(train_dataset, train_sub_indices)

    val_sub_indices = indices[val_start_index:val_start_index+val_use_len]
    val_subset = Subset(train_dataset, val_sub_indices)
    train_dataloader = DataLoader(train_subset, batch_size=batch,
                                shuffle=True)

    val_dataloader = DataLoader(val_subset, batch_size=batch,
                            shuffle=False)

    return train_dataloader, val_dataloader

In [41]:
# Training configurations
seed=5
BATCH_SIZE=32
SLIDE_WIN=5
dim=64
out_layer_num=1
SLIDE_STRIDE=1
topk=5
out_layer_inter_dim=128
val_ratio=0.2
decay=0

In [42]:
cfg = {
            'slide_win': SLIDE_WIN,
            'slide_stride': SLIDE_STRIDE,
        }

In [44]:
import torch
from torch.utils.data import Dataset, DataLoader

import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np


class TimeDataset(Dataset):
    def __init__(self, raw_data, edge_index, mode='train', config = None):
        self.raw_data = raw_data

        self.config = config
        self.edge_index = edge_index
        self.mode = mode

        x_data = raw_data[:-1]
        labels = raw_data[-1]


        data = x_data

        # to tensor
        data = torch.tensor(data).double()
        labels = torch.tensor(labels).double()

        self.x, self.y, self.labels = self.process(data, labels)
    
    def __len__(self):
        return len(self.x)


    def process(self, data, labels):
        x_arr, y_arr = [], []
        labels_arr = []

        slide_win, slide_stride = [self.config[k] for k
            in ['slide_win', 'slide_stride']
        ]
        is_train = self.mode == 'train'

        node_num, total_time_len = data.shape

        rang = range(slide_win, total_time_len, slide_stride) if is_train else range(slide_win, total_time_len)
        
        for i in rang:

            ft = data[:, i-slide_win:i]
            tar = data[:, i]

            x_arr.append(ft)
            y_arr.append(tar)

            labels_arr.append(labels[i])


        x = torch.stack(x_arr).contiguous()
        y = torch.stack(y_arr).contiguous()

        labels = torch.Tensor(labels_arr).contiguous()
        
        return x, y, labels

    def __getitem__(self, idx):

        feature = self.x[idx].double()
        y = self.y[idx].double()

        edge_index = self.edge_index.long()

        label = self.labels[idx].double()

        return feature, y, label, edge_index

In [45]:
train_dataset = TimeDataset(train_dataset_indata, fc_edge_index, mode='train', config=cfg)
test_dataset = TimeDataset(test_dataset_indata, fc_edge_index, mode='test', config=cfg)

In [53]:
seed

5

In [70]:
import matplotlib.pyplot as plt

import json
import random
from torch.utils.data import DataLoader, random_split, Subset

In [69]:
train_dataloader, val_dataloader = get_loaders(train_dataset, seed, BATCH_SIZE, val_ratio = val_ratio)

In [71]:
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                            shuffle=False, num_workers=0)

In [72]:
edge_index_sets = []
edge_index_sets.append(fc_edge_index)

In [76]:
import torch.nn as nn
import time
from torch_geometric.nn import GCNConv, GATConv, EdgeConv
import math
import torch.nn.functional as F

In [77]:
import torch
from torch.nn import Parameter, Linear, Sequential, BatchNorm1d, ReLU
import torch.nn.functional as F
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

from torch_geometric.nn.inits import glorot, zeros
import time
import math

class GraphLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, heads=1, concat=True,
                 negative_slope=0.2, dropout=0, bias=True, inter_dim=-1,**kwargs):
        super(GraphLayer, self).__init__(aggr='add', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.concat = concat
        self.negative_slope = negative_slope
        self.dropout = dropout

        self.__alpha__ = None

        self.lin = Linear(in_channels, heads * out_channels, bias=False)

        self.att_i = Parameter(torch.Tensor(1, heads, out_channels))
        self.att_j = Parameter(torch.Tensor(1, heads, out_channels))
        self.att_em_i = Parameter(torch.Tensor(1, heads, out_channels))
        self.att_em_j = Parameter(torch.Tensor(1, heads, out_channels))

        if bias and concat:
            self.bias = Parameter(torch.Tensor(heads * out_channels))
        elif bias and not concat:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.lin.weight)
        glorot(self.att_i)
        glorot(self.att_j)
        
        zeros(self.att_em_i)
        zeros(self.att_em_j)

        zeros(self.bias)



    def forward(self, x, edge_index, embedding, return_attention_weights=False):
        """"""
        if torch.is_tensor(x):
            x = self.lin(x)
            x = (x, x)
        else:
            x = (self.lin(x[0]), self.lin(x[1]))

        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index,
                                       num_nodes=x[1].size(self.node_dim))

        out = self.propagate(edge_index, x=x, embedding=embedding, edges=edge_index,
                             return_attention_weights=return_attention_weights)

        if self.concat:
            out = out.view(-1, self.heads * self.out_channels)
        else:
            out = out.mean(dim=1)

        if self.bias is not None:
            out = out + self.bias

        if return_attention_weights:
            alpha, self.__alpha__ = self.__alpha__, None
            return out, (edge_index, alpha)
        else:
            return out

    def message(self, x_i, x_j, edge_index_i, size_i,
                embedding,
                edges,
                return_attention_weights):

        x_i = x_i.view(-1, self.heads, self.out_channels)
        x_j = x_j.view(-1, self.heads, self.out_channels)

        if embedding is not None:
            embedding_i, embedding_j = embedding[edge_index_i], embedding[edges[0]]
            embedding_i = embedding_i.unsqueeze(1).repeat(1,self.heads,1)
            embedding_j = embedding_j.unsqueeze(1).repeat(1,self.heads,1)

            key_i = torch.cat((x_i, embedding_i), dim=-1)
            key_j = torch.cat((x_j, embedding_j), dim=-1)



        cat_att_i = torch.cat((self.att_i, self.att_em_i), dim=-1)
        cat_att_j = torch.cat((self.att_j, self.att_em_j), dim=-1)

        alpha = (key_i * cat_att_i).sum(-1) + (key_j * cat_att_j).sum(-1)


        alpha = alpha.view(-1, self.heads, 1)


        alpha = F.leaky_relu(alpha, self.negative_slope)
        alpha = softmax(alpha, edge_index_i, size_i)

        if return_attention_weights:
            self.__alpha__ = alpha

        alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        
        return x_j * alpha.view(-1, self.heads, 1)



    def __repr__(self):
        return '{}({}, {}, heads={})'.format(self.__class__.__name__,
                                             self.in_channels,
                                             self.out_channels, self.heads)

In [82]:
def get_batch_edge_index(org_edge_index, batch_num, node_num):
    # org_edge_index:(2, edge_num)
    edge_index = org_edge_index.clone().detach()
    edge_num = org_edge_index.shape[1]
    batch_edge_index = edge_index.repeat(1,batch_num).contiguous()

    for i in range(batch_num):
        batch_edge_index[:, i*edge_num:(i+1)*edge_num] += i*node_num

    return batch_edge_index.long()


class OutLayer(nn.Module):
    def __init__(self, in_num, node_num, layer_num, inter_num = 512):
        super(OutLayer, self).__init__()

        modules = []

        for i in range(layer_num):
            # last layer, output shape:1
            if i == layer_num-1:
                modules.append(nn.Linear( in_num if layer_num == 1 else inter_num, 1))
            else:
                layer_in_num = in_num if i == 0 else inter_num
                modules.append(nn.Linear( layer_in_num, inter_num ))
                modules.append(nn.BatchNorm1d(inter_num))
                modules.append(nn.ReLU())

        self.mlp = nn.ModuleList(modules)

    def forward(self, x):
        out = x

        for mod in self.mlp:
            if isinstance(mod, nn.BatchNorm1d):
                out = out.permute(0,2,1)
                out = mod(out)
                out = out.permute(0,2,1)
            else:
                out = mod(out)

        return out



class GNNLayer(nn.Module):
    def __init__(self, in_channel, out_channel, inter_dim=0, heads=1, node_num=100):
        super(GNNLayer, self).__init__()


        self.gnn = GraphLayer(in_channel, out_channel, inter_dim=inter_dim, heads=heads, concat=False)

        self.bn = nn.BatchNorm1d(out_channel)
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU()

    def forward(self, x, edge_index, embedding=None, node_num=0):

        out, (new_edge_index, att_weight) = self.gnn(x, edge_index, embedding, return_attention_weights=True)
        self.att_weight_1 = att_weight
        self.edge_index_1 = new_edge_index
  
        out = self.bn(out)
        
        return self.relu(out)


class GDN(nn.Module):
    def __init__(self, edge_index_sets, node_num, dim=64, out_layer_inter_dim=256, input_dim=10, out_layer_num=1, topk=20):

        super(GDN, self).__init__()

        self.edge_index_sets = edge_index_sets

        device = get_device()

        edge_index = edge_index_sets[0]


        embed_dim = dim
        self.embedding = nn.Embedding(node_num, embed_dim)
        self.bn_outlayer_in = nn.BatchNorm1d(embed_dim)


        edge_set_num = len(edge_index_sets)
        self.gnn_layers = nn.ModuleList([
            GNNLayer(input_dim, dim, inter_dim=dim+embed_dim, heads=1) for i in range(edge_set_num)
        ])


        self.node_embedding = None
        self.topk = topk
        self.learned_graph = None

        self.out_layer = OutLayer(dim*edge_set_num, node_num, out_layer_num, inter_num = out_layer_inter_dim)

        self.cache_edge_index_sets = [None] * edge_set_num
        self.cache_embed_index = None

        self.dp = nn.Dropout(0.2)

        self.init_params()
    
    def init_params(self):
        nn.init.kaiming_uniform_(self.embedding.weight, a=math.sqrt(5))


    def forward(self, data, org_edge_index):

        x = data.clone().detach()
        edge_index_sets = self.edge_index_sets

        device = data.device

        batch_num, node_num, all_feature = x.shape
        x = x.view(-1, all_feature).contiguous()


        gcn_outs = []
        for i, edge_index in enumerate(edge_index_sets):
            edge_num = edge_index.shape[1]
            cache_edge_index = self.cache_edge_index_sets[i]

            if cache_edge_index is None or cache_edge_index.shape[1] != edge_num*batch_num:
                self.cache_edge_index_sets[i] = get_batch_edge_index(edge_index, batch_num, node_num).to(device)
            
            batch_edge_index = self.cache_edge_index_sets[i]
            
            all_embeddings = self.embedding(torch.arange(node_num).to(device))

            weights_arr = all_embeddings.detach().clone()
            all_embeddings = all_embeddings.repeat(batch_num, 1)

            weights = weights_arr.view(node_num, -1)

            cos_ji_mat = torch.matmul(weights, weights.T)
            normed_mat = torch.matmul(weights.norm(dim=-1).view(-1,1), weights.norm(dim=-1).view(1,-1))
            cos_ji_mat = cos_ji_mat / normed_mat

            dim = weights.shape[-1]
            topk_num = self.topk

            topk_indices_ji = torch.topk(cos_ji_mat, topk_num, dim=-1)[1]

            self.learned_graph = topk_indices_ji

            gated_i = torch.arange(0, node_num).T.unsqueeze(1).repeat(1, topk_num).flatten().to(device).unsqueeze(0)
            gated_j = topk_indices_ji.flatten().unsqueeze(0)
            gated_edge_index = torch.cat((gated_j, gated_i), dim=0)

            batch_gated_edge_index = get_batch_edge_index(gated_edge_index, batch_num, node_num).to(device)
            gcn_out = self.gnn_layers[i](x, batch_gated_edge_index, node_num=node_num*batch_num, embedding=all_embeddings)

            
            gcn_outs.append(gcn_out)

        x = torch.cat(gcn_outs, dim=1)
        x = x.view(batch_num, node_num, -1)


        indexes = torch.arange(0,node_num).to(device)
        out = torch.mul(x, self.embedding(indexes))
        
        out = out.permute(0,2,1)
        out = F.relu(self.bn_outlayer_in(out))
        out = out.permute(0,2,1)

        out = self.dp(out)
        out = self.out_layer(out)
        out = out.view(-1, node_num)
   

        return out

In [84]:
def get_device():
    # return torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    return 'cpu'

In [85]:
model = GDN(edge_index_sets, len(feature_map), 
                dim=dim, 
                input_dim=SLIDE_WIN,
                out_layer_num=out_layer_num,
                out_layer_inter_dim=out_layer_inter_dim,
                topk=topk
            ).to('cpu')

In [86]:
# util functions about data

from scipy.stats import rankdata, iqr, trim_mean
from sklearn.metrics import f1_score, mean_squared_error
import numpy as np
from numpy import percentile


def get_attack_interval(attack): 
    heads = []
    tails = []
    for i in range(len(attack)):
        if attack[i] == 1:
            if attack[i-1] == 0:
                heads.append(i)
            
            if i < len(attack)-1 and attack[i+1] == 0:
                tails.append(i)
            elif i == len(attack)-1:
                tails.append(i)
    res = []
    for i in range(len(heads)):
        res.append((heads[i], tails[i]))
    # print(heads, tails)
    return res

# calculate F1 scores
def eval_scores(scores, true_scores, th_steps, return_thresold=False):
    padding_list = [0]*(len(true_scores) - len(scores))
    # print(padding_list)

    if len(padding_list) > 0:
        scores = padding_list + scores

    scores_sorted = rankdata(scores, method='ordinal')
    th_steps = th_steps
    # th_steps = 500
    th_vals = np.array(range(th_steps)) * 1.0 / th_steps
    fmeas = [None] * th_steps
    thresholds = [None] * th_steps
    for i in range(th_steps):
        cur_pred = scores_sorted > th_vals[i] * len(scores)

        fmeas[i] = f1_score(true_scores, cur_pred)

        score_index = scores_sorted.tolist().index(int(th_vals[i] * len(scores)+1))
        thresholds[i] = scores[score_index]

    if return_thresold:
        return fmeas, thresholds
    return fmeas

def eval_mseloss(predicted, ground_truth):

    ground_truth_list = np.array(ground_truth)
    predicted_list = np.array(predicted)

    
    # mask = (ground_truth_list == 0) | (predicted_list == 0)

    # ground_truth_list = ground_truth_list[~mask]
    # predicted_list = predicted_list[~mask]

    # neg_mask = predicted_list < 0
    # predicted_list[neg_mask] = 0

    # err = np.abs(predicted_list / ground_truth_list - 1)
    # acc = (1 - np.mean(err))

    # return loss
    loss = mean_squared_error(predicted_list, ground_truth_list)

    return loss

def get_err_median_and_iqr(predicted, groundtruth):

    np_arr = np.abs(np.subtract(np.array(predicted), np.array(groundtruth)))

    err_median = np.median(np_arr)
    err_iqr = iqr(np_arr)

    return err_median, err_iqr

def get_err_median_and_quantile(predicted, groundtruth, percentage):

    np_arr = np.abs(np.subtract(np.array(predicted), np.array(groundtruth)))

    err_median = np.median(np_arr)
    # err_iqr = iqr(np_arr)
    err_delta = percentile(np_arr, int(percentage*100)) - percentile(np_arr, int((1-percentage)*100))

    return err_median, err_delta

def get_err_mean_and_quantile(predicted, groundtruth, percentage):

    np_arr = np.abs(np.subtract(np.array(predicted), np.array(groundtruth)))

    err_median = trim_mean(np_arr, percentage)
    # err_iqr = iqr(np_arr)
    err_delta = percentile(np_arr, int(percentage*100)) - percentile(np_arr, int((1-percentage)*100))

    return err_median, err_delta

def get_err_mean_and_std(predicted, groundtruth):

    np_arr = np.abs(np.subtract(np.array(predicted), np.array(groundtruth)))

    err_mean = np.mean(np_arr)
    err_std = np.std(np_arr)

    return err_mean, err_std


def get_f1_score(scores, gt, contamination):

    padding_list = [0]*(len(gt) - len(scores))
    # print(padding_list)

    threshold = percentile(scores, 100 * (1 - contamination))

    if len(padding_list) > 0:
        scores = padding_list + scores

    pred_labels = (scores > threshold).astype('int').ravel()

    return f1_score(gt, pred_labels)

In [87]:
# Evaluate Methods
import numpy as np
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score


def get_full_err_scores(test_result, val_result):
    np_test_result = np.array(test_result)
    np_val_result = np.array(val_result)

    all_scores =  None
    all_normals = None
    feature_num = np_test_result.shape[-1]

    labels = np_test_result[2, :, 0].tolist()

    for i in range(feature_num):
        test_re_list = np_test_result[:2,:,i]
        val_re_list = np_val_result[:2,:,i]

        scores = get_err_scores(test_re_list, val_re_list)
        normal_dist = get_err_scores(val_re_list, val_re_list)

        if all_scores is None:
            all_scores = scores
            all_normals = normal_dist
        else:
            all_scores = np.vstack((
                all_scores,
                scores
            ))
            all_normals = np.vstack((
                all_normals,
                normal_dist
            ))

    return all_scores, all_normals


def get_final_err_scores(test_result, val_result):
    full_scores, all_normals = get_full_err_scores(test_result, val_result, return_normal_scores=True)

    all_scores = np.max(full_scores, axis=0)

    return all_scores



def get_err_scores(test_res, val_res):
    test_predict, test_gt = test_res
    val_predict, val_gt = val_res

    n_err_mid, n_err_iqr = get_err_median_and_iqr(test_predict, test_gt)

    test_delta = np.abs(np.subtract(
                        np.array(test_predict).astype(np.float64), 
                        np.array(test_gt).astype(np.float64)
                    ))
    epsilon=1e-2

    err_scores = (test_delta - n_err_mid) / ( np.abs(n_err_iqr) +epsilon)

    smoothed_err_scores = np.zeros(err_scores.shape)
    before_num = 3
    for i in range(before_num, len(err_scores)):
        smoothed_err_scores[i] = np.mean(err_scores[i-before_num:i+1])

    
    return smoothed_err_scores



def get_loss(predict, gt):
    return eval_mseloss(predict, gt)

def get_f1_scores(total_err_scores, gt_labels, topk=1):
    print('total_err_scores', total_err_scores.shape)
    # remove the highest and lowest score at each timestep
    total_features = total_err_scores.shape[0]

    # topk_indices = np.argpartition(total_err_scores, range(total_features-1-topk, total_features-1), axis=0)[-topk-1:-1]
    topk_indices = np.argpartition(total_err_scores, range(total_features-topk-1, total_features), axis=0)[-topk:]
    
    topk_indices = np.transpose(topk_indices)

    total_topk_err_scores = []
    topk_err_score_map=[]
    # topk_anomaly_sensors = []

    for i, indexs in enumerate(topk_indices):
       
        sum_score = sum( score for k, score in enumerate(sorted([total_err_scores[index, i] for j, index in enumerate(indexs)])) )

        total_topk_err_scores.append(sum_score)

    final_topk_fmeas = eval_scores(total_topk_err_scores, gt_labels, 400)

    return final_topk_fmeas

def get_val_performance_data(total_err_scores, normal_scores, gt_labels, topk=1):
    total_features = total_err_scores.shape[0]

    topk_indices = np.argpartition(total_err_scores, range(total_features-topk-1, total_features), axis=0)[-topk:]

    total_topk_err_scores = []
    topk_err_score_map=[]

    total_topk_err_scores = np.sum(np.take_along_axis(total_err_scores, topk_indices, axis=0), axis=0)

    thresold = np.max(normal_scores)

    pred_labels = np.zeros(len(total_topk_err_scores))
    pred_labels[total_topk_err_scores > thresold] = 1

    for i in range(len(pred_labels)):
        pred_labels[i] = int(pred_labels[i])
        gt_labels[i] = int(gt_labels[i])

    pre = precision_score(gt_labels, pred_labels)
    rec = recall_score(gt_labels, pred_labels)

    f1 = f1_score(gt_labels, pred_labels)


    auc_score = roc_auc_score(gt_labels, total_topk_err_scores)

    return f1, pre, rec, auc_score, thresold


def get_best_performance_data(total_err_scores, gt_labels, topk=1):

    total_features = total_err_scores.shape[0]

    # topk_indices = np.argpartition(total_err_scores, range(total_features-1-topk, total_features-1), axis=0)[-topk-1:-1]
    topk_indices = np.argpartition(total_err_scores, range(total_features-topk-1, total_features), axis=0)[-topk:]

    total_topk_err_scores = []
    topk_err_score_map=[]

    total_topk_err_scores = np.sum(np.take_along_axis(total_err_scores, topk_indices, axis=0), axis=0)

    final_topk_fmeas ,thresolds = eval_scores(total_topk_err_scores, gt_labels, 400, return_thresold=True)

    th_i = final_topk_fmeas.index(max(final_topk_fmeas))
    thresold = thresolds[th_i]

    pred_labels = np.zeros(len(total_topk_err_scores))
    pred_labels[total_topk_err_scores > thresold] = 1

    for i in range(len(pred_labels)):
        pred_labels[i] = int(pred_labels[i])
        gt_labels[i] = int(gt_labels[i])

    pre = precision_score(gt_labels, pred_labels)
    rec = recall_score(gt_labels, pred_labels)

    auc_score = roc_auc_score(gt_labels, total_topk_err_scores)

    return max(final_topk_fmeas), pre, rec, auc_score, thresold

In [88]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import time
from sklearn.metrics import mean_squared_error
import torch.nn.functional as F
import numpy as np

from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score
from torch.utils.data import DataLoader, random_split, Subset
from scipy.stats import iqr

In [102]:
# Train methods
def loss_func(y_pred, y_true):
    loss = F.mse_loss(y_pred, y_true, reduction='mean')

    return loss



def train(model = None, save_path = '', config={},  train_dataloader=None, val_dataloader=None, feature_map={}, test_dataloader=None, test_dataset=None, dataset_name='swat', train_dataset=None):

    seed = config['seed']

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=config['decay'])

    now = time.time()
    
    train_loss_list = []
    cmp_loss_list = []

    device = get_device()


    acu_loss = 0
    min_loss = 1e+8
    min_f1 = 0
    min_pre = 0
    best_prec = 0

    i = 0
    epoch = config['epoch']
    early_stop_win = 15

    model.train()

    log_interval = 1000
    stop_improve_count = 0

    dataloader = train_dataloader

    for i_epoch in range(epoch):

        acu_loss = 0
        model.train()

        for x, labels, attack_labels, edge_index in dataloader:
            _start = time.time()

            x, labels, edge_index = [item.float().to(device) for item in [x, labels, edge_index]]

            optimizer.zero_grad()
            print(x.shape)
            print(edge_index.shape)
            out = model(x, edge_index).float().to(device)
            loss = loss_func(out, labels)
            
            loss.backward()
            optimizer.step()

            
            train_loss_list.append(loss.item())
            acu_loss += loss.item()
                
            i += 1


        # each epoch
        print('epoch ({} / {}) (Loss:{:.8f}, ACU_loss:{:.8f})'.format(
                        i_epoch, epoch, 
                        acu_loss/len(dataloader), acu_loss), flush=True
            )

        # use val dataset to judge
        if val_dataloader is not None:

            val_loss, val_result = test(model, val_dataloader)

            if val_loss < min_loss:
                torch.save(model.state_dict(), save_path)

                min_loss = val_loss
                stop_improve_count = 0
            else:
                stop_improve_count += 1


            if stop_improve_count >= early_stop_win:
                break

        else:
            if acu_loss < min_loss :
                torch.save(model.state_dict(), save_path)
                min_loss = acu_loss



    return train_loss_list

In [91]:
train_config = {
        'batch': BATCH_SIZE,
        'epoch': 30,
        'slide_win': SLIDE_WIN,
        'dim': dim,
        'slide_stride': SLIDE_STRIDE,
        'comment': "TELSTRA DATA",
        'seed': seed,
        'out_layer_num': out_layer_num,
        'out_layer_inter_dim': out_layer_inter_dim,
        'decay': decay,
        'val_ratio': val_ratio,
        'topk': topk,
    }

In [94]:
env_config={
        'save_path': '',
        'dataset': "TELSTRA DATA",
        'report': 'best',
        'device': 'cpu',
        'load_model_path': ''
    }

In [95]:
def get_score(test_result, val_result):

        feature_num = len(test_result[0][0])
        np_test_result = np.array(test_result)
        np_val_result = np.array(val_result)

        test_labels = np_test_result[2, :, 0].tolist()
    
        test_scores, normal_scores = get_full_err_scores(test_result, val_result)

        top1_best_info = get_best_performance_data(test_scores, test_labels, topk=1) 
        top1_val_info = get_val_performance_data(test_scores, normal_scores, test_labels, topk=1)


        print('=========================** Result **============================\n')

        info = None
        if env_config['report'] == 'best':
            info = top1_best_info
        elif env_config['report'] == 'val':
            info = top1_val_info

        print(f'F1 score: {info[0]}')
        print(f'precision: {info[1]}')
        print(f'recall: {info[2]}\n')

In [98]:
def run():

        model_save_path = ''

        self.train_log = train(model, model_save_path, 
            config = train_config,
            train_dataloader=train_dataloader,
            val_dataloader=val_dataloader, 
            feature_map=feature_map,
            test_dataloader=test_dataloader,
            test_dataset=test_dataset,
            train_dataset=train_dataset,
            dataset_name=env_config['dataset']
        )
        
        # test            
        self.model.load_state_dict(torch.load(model_save_path))
        best_model = model.to(device)

        _, self.test_result = test(best_model, test_dataloader)
        _, self.val_result = test(best_model, val_dataloader)

        get_score(test_result, val_result)

In [103]:
run()

torch.Size([32, 18, 5])
torch.Size([32, 2, 306])


RuntimeError: The expanded size of the tensor (1) must match the existing size (2880) at non-singleton dimension 1.  Target sizes: [2880, 1, 64].  Tensor sizes: [1, 2880, 1]